Dataset Information :
1. id - Unique ID for a news article
2. title - The Title of the news article
3. author - Author of the news article
4. text - Text of the article, could be incomplete
5. label - Label that marks that the news article is fake or real

        1: Fake News
        0: Real News

In [24]:
# IMPORTING THE MODULES
import numpy as np
import pandas as pd
import re     # for searching words in a text or paragraph
from nltk.corpus import stopwords    # words that dont add much value to our paragraph or text
from nltk.stem.porter import PorterStemmer   # gives the root word for the particular word
from sklearn.feature_extraction.text import TfidfVectorizer  # to convert text into feature vectors or numeric data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 

In [25]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\malha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
# PRINTING THE STOPWORDS IN ENGLISH
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [27]:
# LOADING THE DATASET
newsDataset = pd.read_csv('train.csv')

In [28]:
newsDataset.shape

(20800, 5)

In [29]:
newsDataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [30]:
# COUNTING THE NUMBER OF MISSING VALUES IN THE DATASET
newsDataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [31]:
# IF WE HAVE A SMALL NUMBER OF MISSING VALUES THEN WE CAN DROP THE VALUES
# REPLACING THE NULL VALUES WITH EMPTY STRINGS

newsDataset = newsDataset.fillna('')

In [32]:
newsDataset['content'] = newsDataset['author'] + ' ' + newsDataset['title']

In [33]:
print(newsDataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [34]:
newsDataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [35]:
# SEPERATING THE DATA AND LABELS
x = newsDataset.drop(columns='label', axis = 1)
y = newsDataset['label']

print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [36]:
# STEMMING PROCESS
# STEMMING IS THE PROCESS OF REDUCING A WORD TO IT'S ROOT WORD -----  REMOVING PREFIXES AND SUFFIXES
# EXAMPLE :- ACTOR, ACTRESS, ACTING --------> ACT

port_stem = PorterStemmer()

In [37]:
def stemming(content):
    stemmedContent = re.sub('[^a-zA-z]',' ', content) # ---> filters the content and keeps only letters from a-z and replaces any other character by ' '.
    stemmedContent = stemmedContent.lower() # ---> converted to lower case.
    stemmedContent = stemmedContent.split()
    stemmedContent = [port_stem.stem(word) for word in stemmedContent if not word in stopwords.words('english')] # ---> perform stemming on every word in the stemmed content datafram that is not in the stopwords.
    stemmedContent = ' '.join(stemmedContent)  #---> join the words
    return stemmedContent

In [38]:
newsDataset['content'] =newsDataset['content'].apply(stemming)

In [39]:
print(newsDataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [40]:
# seperating the data and label
x = newsDataset['content'].values
y = newsDataset['label']

In [41]:
print(x)
print(y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


In [42]:
y.shape

(20800,)

In [43]:
x.shape

(20800,)

In [45]:
# CONVERTING TEXTUAL DATA INTO NUMERICAL DATA
vectorizer = TfidfVectorizer()
vectorizer.fit(x)

x = vectorizer.transform(x)

In [46]:
print(x)

  (0, 15697)	0.28485063562728646
  (0, 13480)	0.2565896679337957
  (0, 8915)	0.3635963806326075
  (0, 8636)	0.29212514087043684
  (0, 7698)	0.24785219520671603
  (0, 7012)	0.21874169089359144
  (0, 4979)	0.233316966909351
  (0, 3795)	0.2705332480845492
  (0, 3603)	0.3598939188262559
  (0, 2962)	0.2468450128533713
  (0, 2485)	0.3676519686797209
  (0, 268)	0.27010124977708766
  (1, 16812)	0.30071745655510157
  (1, 6823)	0.1904660198296849
  (1, 5510)	0.7143299355715573
  (1, 3571)	0.26373768806048464
  (1, 2816)	0.19094574062359204
  (1, 2224)	0.3827320386859759
  (1, 1895)	0.15521974226349364
  (1, 1498)	0.2939891562094648
  (2, 15622)	0.41544962664721613
  (2, 9625)	0.49351492943649944
  (2, 5975)	0.3474613386728292
  (2, 5396)	0.3866530551182615
  (2, 3106)	0.46097489583229645
  :	:
  (20797, 13128)	0.2482526352197606
  (20797, 12350)	0.27263457663336677
  (20797, 12144)	0.24778257724396507
  (20797, 10311)	0.08038079000566466
  (20797, 9593)	0.174553480255222
  (20797, 9523)	0.295420

In [47]:
# SPLITTING THE DATASET INTO TRAINING AND TEST DATA
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

In [48]:
model = LogisticRegression()

In [49]:
model.fit(x_train, y_train)

LogisticRegression()

In [51]:
# MODEL EVALUATION ON TRAINING DATA
# ACCURACY SCORE

x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)
print("Accuracy score on TRAINING DATA is ", training_data_accuracy)

Accuracy score on TRAINING DATA is  0.9864783653846154


In [52]:
# MODEL EVALUATION ON TESTING DATA
# ACCURACY SCORE

x_test_prediction = model.predict(x_test)
testing_data_accuracy = accuracy_score(x_test_prediction, y_test)
print("Accuracy Score on TESTING DATA is ", testing_data_accuracy)


Accuracy Score on TESTING DATA is  0.979326923076923


In [57]:
# MAKING A PREDICTIVE SYSTEM

x_new = x_test[0]

prediction = model.predict(x_new)

if prediction == 0:
    print("The News is REAL!")
else:
    print("The News is FAKE!")

[1]
The News is FAKE!


In [58]:
# CHECKING THE ANSWER
print(y_test[0])

1
